# Introducing: Feature modifiers

With the current L2 Constructs, all feature abstractions are enclosed within the respective Construct. Take a `Topic` as an example: The L2 includes an abstraction to help with FIFO Topics. It codifies the dependency between some low-level CFN properties and the topic name having to end in `.fifo`. This abstraction is currently *only* available to users of the L2. If I want to use this feature, I *have to* use the L2. This is problematic in scenarios where users are interested in *some*, but not *all* of the features of an L2.

The basic idea behind feature modifiers is to break-out these abstractions from being tightly coupled to an L2. They should be usable with any `Topic`, or - going even further - with any Construct tree that includes `n` `Topics` (with `n >= 1`).

## Basic example: Topic

Taking the aforementioned, the current L2 features can be re-created with these two feature modifiers: `sns.encrypt` to handle encryption and `sns.fifo` to help with FIFO topics.

In [1]:
import './lib/setup.ts';

const topicKey = new kms.Key(stack, "TopicKey");
const sourceTopic = new sns.L1Topic(stack, "IncomingMessages");
sns.encrypt(sourceTopic, topicKey)
sns.fifo(sourceTopic, {
  contentBasedDeduplication: true,
});

print(sourceTopic, topicKey);

TopicKeyB2E0C9CB:
  Type: 'AWS::KMS::Key'
  Properties:
    KeyPolicy:
      Statement:
        - Action: 'kms:*'
          Effect: Allow
          Principal:
            AWS:
              'Fn::Join':
                - ''
                - - 'arn:'
                  - Ref: 'AWS::Partition'
                  - ':iam::'
                  - Ref: 'AWS::AccountId'
                  - ':root'
          Resource: '*'
      Version: '2012-10-17'
  UpdateReplacePolicy: Retain
  DeletionPolicy: Retain
IncomingMessages:
  Type: 'AWS::SNS::Topic'
  Properties:
    kmsMasterKeyId:
      'Fn::GetAtt':
        - TopicKeyB2E0C9CB
        - Arn
    topicName: IncomingMessages.fifo
    fifoTopic: true
    contentBasedDeduplication: true



## Add a Queue as a target for the Topic

To expend our example, we add a `Queue` as a target for a later added `TopicSubscription`. Let's use this new resources to look at possible syntax variations.

### Excursion: Syntax variations

This uses a variation of the modifiers syntax: Instead of directly calling the modifier, we pass it to the `with()` method of the resource construct.s
The downside of it is limited support in code auto-completion. `with()` would accept *any* modifier, so we can't suggest which modifier to use and what parameters the modifier supports. A workaround would be to use overloads in the method definition. With overloads, we get a list of expected modifiers and once the first parameter is provided, the auto-completion appears to be able to pick the right one.

<img src="images/modifiers_with_help.png" width="400" class="blog-image">

Line 8 uses yet another alternative syntax, which improves code auto-completion. These additional methods would have to be codegen'd and would only support built-in modifiers.

<img src="images/modifiers_use_help.png" width="450" class="blog-image">

### Inspired by pipeline operator

The syntax for these is inspired by the *tc39 pipeline operator proposal*. Any variation of it could be considered going forward. Crucially, it is syntactic sugar on top of the base implementation of a function that takes a construct as first parameter. See [this article implementing the pipeline operator proposal](https://dev.to/nexxeln/implementing-the-pipe-operator-in-typescript-30ip) for further information.

In [8]:
const targetQueue = new sqs.L1Queue(stack, "AcceptedMessages")
  .with(sqs.visibilityTimeout, cdk.Duration.seconds(300))
  .with(sqs.enforceSSL)
  .useEncryptWithKey();

print(targetQueue);

AcceptedMessages:
  Type: 'AWS::SQS::Queue'
  Properties:
    visibilityTimeout: 300
    kmsMasterKeyId:
      'Fn::GetAtt':
        - AcceptedMessagesKey1F944073
        - Arn
AcceptedMessagesPolicyA013B031:
  Type: 'AWS::SQS::QueuePolicy'
  Properties:
    PolicyDocument:
      Statement:
        - Action: 'sqs:*'
          Condition:
            Bool:
              'aws:SecureTransport': 'false'
          Effect: Deny
          Principal:
            AWS: '*'
          Resource:
            'Fn::GetAtt':
              - AcceptedMessages
              - Arn
      Version: '2012-10-17'
    Queues:
      - 'Fn::GetAtt':
          - AcceptedMessages
          - QueueUrl
AcceptedMessagesKey1F944073:
  Type: 'AWS::KMS::Key'
  Properties:
    Description: Created by Default/AcceptedMessages
    KeyPolicy:
      Statement:
        - Action: 'kms:*'
          Effect: Allow
          Principal:
            AWS:
              'Fn::Join':
                - ''
                - - 'arn:'
        

### Create a DLQ for the Subscription

`TopicSubscription`s support dead-letter queues for undelivered messages. Let's create an other queue for this purpose, using yet another syntax variation.

### Higher-order programming

So far, the modifier type has been a bit messy. While it generally takes "a thing" as the first parameter, it's wild and undefined what comes after it: Sometimes it's a single value like a `Duration`, at other times we have multiple values (some of which are optional) and even a property bag!

One way to get around this inconsistency, is higher-order programming. With HOP we can strictly define modifiers as taking a single *subject* that executes the modification. Because we still need the ability to pass in values, our "feature modifiers" are now a factory instead: For a given encryption `Key`, make me a feature modifier that encrypts the `Queue` with it.

In [3]:
const dlq = new sqs.L1Queue(stack, "FilteredMessages")
  .with(sqs.encryptWithKeyStrict(new kms.Key(stack, "FilteredQueueKey")));

print(dlq);

{"code":-32000,"message":"Promise was collected"}:  

FilteredMessages:
  Type: 'AWS::SQS::Queue'
  Properties:
    kmsMasterKeyId:
      'Fn::GetAtt':
        - FilteredQueueKey896A5D37
        - Arn



## Vocabulary

We have already established, that modifiers operate on a *subject*. If we lean into this analogy to the English language grammar, we can get inspiration for further terminology. Note this is not aiming to be an accurate mapping to grammar rules. Natural languages is weird and beautiful and ambagious, whereas code should be clear and unambiguous.

**Subject**

The Construct we are operating with. When multiple Constructs are involved, this is the main one. In a localized Construct tree, it would be the parent. Usually these are CFN Resources or L2s: `Topic`, `Queue` but it can be more complex like an `App`.

**Predicate** or **Action**

The modifier itself. It expresses an action or being. `encrypt()` is an example for action, `fifo()` for being. With this in mind, we could change `visibilityTimeout()` to an arguably more friendly `visibleFor()`.

**Adverb**

In natural language, adverbs are used to further modify or describe the predicate. The `contentBasedDeduplication` property is a further description of the `fifo()` being of `Topic`. The duration of `visibleFor()` further modifies how long messages are visible for. Now it becomes more obvious why a higher-order modifier form could be considered as more pure than other variations: Arguments are only used in the context they are relevant for.

**Objects**

Any other Constructs that are part of the action. To `encrypt()` a `Topic` we need a `Key`. So the `Key` becomes the *object*. 

❌ This implementation of `L1Subscription` without receiving a `Topic` and `Queue` at construction time doesn't work. Both `TopicArn` and `Protocol` are required.

In [7]:
const topic2queue = new sub.L1Subscription(stack, "FilterMessages")
  .with(sub.connect(sourceTopic, targetQueue))
  .with(sub.deadLetterQueue(dlq))
  .with(sub.filterMessages({
    color: sns.SubscriptionFilter.stringFilter({
      allowlist: ["red", "orange"],
      matchPrefixes: ["bl"],
    }),
  }));

print(topic2queue, targetQueue.policy);

{"code":-32000,"message":"Promise was collected"}:  

AcceptedMessagesPolicyA013B031:
  Type: 'AWS::SQS::QueuePolicy'
  Properties:
    PolicyDocument:
      Statement:
        - Action: 'sqs:*'
          Condition:
            Bool:
              'aws:SecureTransport': 'false'
          Effect: Deny
          Principal:
            AWS: '*'
          Resource:
            'Fn::GetAtt':
              - AcceptedMessages
              - Arn
        - Action: 'sqs:SendMessage'
          Condition:
            ArnEquals: {}
          Effect: Allow
          Principal:
            Service: sns.amazonaws.com
          Resource:
            'Fn::GetAtt':
              - AcceptedMessages
              - Arn
      Version: '2012-10-17'
    Queues:
      - 'Fn::GetAtt':
          - AcceptedMessages
          - QueueUrl
FilterMessages:
  Type: 'AWS::SNS::Subscription'
  Properties:
    endpoint:
      'Fn::GetAtt':
        - AcceptedMessages
        - Arn
    protocol: sqs
    redrivePolicy:
      deadLetterTargetArn:
        'Fn::GetAtt':
         

In [9]:
// Apply modifier to any construct tree
sqs.enforceSSL(stack)

// Or maybe this?
const queueVisibleFor = any(sqs.L1Queue.isL1Queue, sqs.visibilityTimeoutStrict(cdk.Duration.seconds(500)));
queueVisibleFor(stack);

printOnly(targetQueue);
print(dlq);

AcceptedMessages:
  Type: 'AWS::SQS::Queue'
  Properties:
    visibilityTimeout: 500
    kmsMasterKeyId:
      'Fn::GetAtt':
        - AcceptedMessagesKey1F944073
        - Arn

FilteredMessages:
  Type: 'AWS::SQS::Queue'
  Properties:
    kmsMasterKeyId:
      'Fn::GetAtt':
        - FilteredQueueKey896A5D37
        - Arn
    visibilityTimeout: 500
FilteredMessagesPolicy918142A1:
  Type: 'AWS::SQS::QueuePolicy'
  Properties:
    PolicyDocument:
      Statement:
        - Action: 'sqs:*'
          Condition:
            Bool:
              'aws:SecureTransport': 'false'
          Effect: Deny
          Principal:
            AWS: '*'
          Resource:
            'Fn::GetAtt':
              - FilteredMessages
              - Arn
        - Action: 'sqs:SendMessage'
          Condition:
            ArnEquals: {}
          Effect: Allow
          Principal:
            Service: sns.amazonaws.com
          Resource:
            'Fn::GetAtt':
              - FilteredMessages
             